In [11]:
import gradio as gr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

In [19]:
def search_data(sku, group_name, date):
    
    
    if(sku == '' and group_name == '' and date != ''):
        filtered_df = initial_df[initial_df['Date'] == date]
    
    elif(sku == '' and group_name != '' and date == ''):
        filtered_df = initial_df[initial_df['Group Name'] == group_name]
        
    elif(sku != '' and group_name == '' and date == ''):
        filtered_df = initial_df[initial_df['Current Job'] == sku]
    
    elif(sku == '' and group_name != '' and date != ''):
        filtered_df = initial_df[(initial_df['Group Name'] == group_name) & (initial_df['Date'] == date)]
        
    elif(sku != '' and group_name == '' and date != ''):
        filtered_df = initial_df[(initial_df['Current Job'] == sku) & (initial_df['Date'] == date)]
        
    elif(sku != '' and group_name != '' and date == ''):
        filtered_df = initial_df[(initial_df['Current Job'] == sku) & (initial_df['Group Name'] == group_name)]
    
    else:
        filtered_df = initial_df[(initial_df['Current Job'] == sku) & (initial_df['Group Name'] == group_name) & (initial_df['Date'] == date)]
    
    
    records = len(filtered_df)
    
    compliant_records = len(filtered_df[(filtered_df['Units Per Hour'] >= filtered_df['Lower Target']) & (filtered_df['Units Per Hour'] <= filtered_df['Upper Target'])])
    
    compliance_percentage = round((compliant_records / records),2) * 100 if records != 0 else 0
    
    avg_units_per_hour = round(filtered_df['Units Per Hour'].mean(),2)
    
    stdev_units_per_hour = round(filtered_df['Units Per Hour'].std(),2)
    
    most_common_value = filtered_df['Description'].value_counts().idxmax()

    print(most_common_value)
    
    fig1 = plt.figure()
    plt.hist(filtered_df['Units Per Hour'], bins=20, color='skyblue', edgecolor='black')
    plt.xlabel('Units Per Hour')
    plt.ylabel('Frequency')
    plt.grid(True)
    
    return most_common_value, records, compliance_percentage, avg_units_per_hour, stdev_units_per_hour, fig1




def upload_file(filepath):
    global initial_df
    name = Path(filepath).name
    initial_df = prepare_df(filepath)
    initial_analysis_output = initial_analysis(initial_df)
    return initial_analysis_output

def initial_analysis(df):
    
    compliant_df = df[(df['Units Per Hour'] >= df['Lower Target']) & (df['Units Per Hour'] <= df['Upper Target'])]
    non_compliant_df = df[(df['Units Per Hour'] < df['Lower Target']) | (df['Units Per Hour'] > df['Upper Target'])]

    total_observations = len(df)
    pct_compliant = 100 * round(len(compliant_df)/total_observations,2)
    pct_non_compliant = 100 * round(len(non_compliant_df)/total_observations,2)
    
    return total_observations, pct_compliant, pct_non_compliant

def prepare_df(file_path):

    df = pd.read_excel(file_path)
    
    df['Current Job'] = df['Current Job'].astype(str)

    df['Start of Batch Date/Time'] = pd.to_datetime(df['Start of Batch Date/Time'])
    df['End of Batch Date/Time'] = pd.to_datetime(df['End of Batch Date/Time'])

    df['Batch Length (Hours)'] = (df['End of Batch Date/Time'] - df['Start of Batch Date/Time']).dt.total_seconds() / 3600

    df['Units Per Hour'] = df['Batch Count'] / df['Batch Length (Hours)']

    df.dropna(subset=['Optimal Cases Per Hour'], inplace=True)

    return df


tab_search_by_sku = gr.Interface(
    fn = search_data,
    inputs = [
        gr.Textbox(label="SKU"), 
        gr.Textbox(label="Group Name"),
        gr.Textbox(label="Date", info = "yyyy-mm-dd"), 
    ],
    outputs = [
        gr.Textbox(label="Description"),
        gr.Textbox(label="Records"), 
        gr.Textbox(label="Compliance %"),
        gr.Textbox(label="Avg Units/Hr"), 
        gr.Textbox(label="Std Dev Units/Hr"), 
        gr.Plot(label="Units Per Hour Distribution")
    ],
    allow_flagging = "never"
)

tab_files = gr.Interface(
    fn = upload_file,
    inputs = [
        gr.File(label="Upload Template File", file_types=[".xlsx"], file_count="single")
    ],
    outputs = [
        gr.Textbox(label="Total Observations", interactive=False), 
        gr.Textbox(label="Compliant Observations %", interactive=False),
        gr.Textbox(label="Non-Compliant Observations %", interactive=False)
    ],
    allow_flagging = "never"
    
)

# tab_files = gr.Interface(
#     fn = upload_file,
#     inputs = [
#         gr.UploadButton("Click to Upload Template File", file_types=["files"], file_count="single")
#     ],
#     outputs = [
#         gr.Textbox(label="Total Observations", interactive=False), 
#         gr.Textbox(label="Compliant Observations %", interactive=False),
#         gr.Textbox(label="Non-Compliant Observations %", interactive=False)
#     ],
#     allow_flagging = "never"
    
# )
    
demo = gr.TabbedInterface([tab_files, tab_search_by_sku], ["Upload Sheet", "Search"])

demo.launch(share=True, auth=("BrianA", "Brian@Muffintown1"))


Running on local URL:  http://127.0.0.1:7895
Running on public URL: https://699455bcbcb2308306.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/Users/abdurrahman/anaconda3/lib/python3.11/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


2167 Muffin, WG, Choc Chip, IW
